In [1]:
import numpy as np
import scipy.optimize as opt
import scipy.stats as sts
import pandas as pd

dt = pd.read_csv('MacroSeries.txt', sep = ',', header = None, names = ['c','k','w','r'])

In [2]:
# 1(a)
def z_wk(w,k,a):
    z = np.log(w) - np.log(1-a) - a*np.log(k)
    return z

def z_pdf(z, rho, mu, sig):
    zlag = z.shift(1)
    zlag[0] = mu
    pdf_val = (1/(sig*np.sqrt(2*np.pi)) * np.exp(-(z-rho*zlag-(1-rho)*mu)**2/(2*sig**2)))
    return pdf_val

def log_lik(z, rho, mu, sig):
    pdf_val = z_pdf(z, rho, mu, sig)
    log_pdf_val = np.log(pdf_val)
    log_lik = log_pdf_val.sum()
    return log_lik

def crit_wk(params, *args):
    a, rho, mu, sig = params
    w, k = args
    z = z_wk(w, k, a)
    neg_log_lik = -log_lik(z, rho, mu, sig)
    return neg_log_lik

a = 0.3
rho = 0.9
mu = 10
sig = 0.15

params_init = np.array([a,rho,mu,sig])
mle_args = (dt['w'], dt['k'])
params_bound = ((1e-10,1-1e-10), (-1+1e-10, 1-1e-10), (1e-10,None), (1e-10,None))

result = opt.minimize(crit_wk, params_init, args = (mle_args), bounds = params_bound, method = 'L-BFGS-B')
a_mle, rho_mle, mu_mle, sig_mle = result.x
print('alpha MLE =', a_mle, '; rho MLE =', rho_mle, '; mu MLE =', mu_mle, '; sigma MLE =', sig_mle)
print('Inverse Hessian Matrix =')
print(result.hess_inv*np.identity(4))

alpha MLE = 0.457529974922 ; rho MLE = 0.720478914862 ; mu MLE = 9.52252983019 ; sigma MLE = 0.091996603229
Inverse Hessian Matrix =
[[  7.53348058e+02  -5.44925858e+02  -1.05852485e+04   2.00850197e+01]
 [ -5.44925858e+02   3.97118761e+02   7.65472161e+03  -1.41553267e+01]
 [ -1.05852485e+04   7.65472161e+03   1.48734049e+05  -2.82466417e+02]
 [  2.00850197e+01  -1.41553267e+01  -2.82466417e+02   5.82749219e-01]]


In [3]:
# 1(b)
def z_rk(r,k,a):
    z = np.log(r) - np.log(a) + (1-a)*np.log(k)
    return z

def crit_rk(params, *args):
    a, rho, mu, sig = params
    r, k = args
    z = z_rk(r, k, a)
    neg_log_lik = -log_lik(z, rho, mu, sig)
    return neg_log_lik

params_init1 = np.array([0.3,0.9,12,0.15])
mle_args1 = (dt['r'], dt['k'])

result1 = opt.minimize(crit_rk, params_init1, args = (mle_args1), bounds = params_bound, method = 'L-BFGS-B')
a_mle1, rho_mle1, mu_mle1, sig_mle1 = result1.x
print('alpha MLE =', a_mle1, '; rho MLE =', rho_mle1, '; mu MLE =', mu_mle1, '; sigma MLE =', sig_mle1)
print('Inverse Hessian Matrix =')
print(result1.hess_inv*np.identity(4))

alpha MLE = 0.457527569204 ; rho MLE = 0.720478308403 ; mu MLE = 9.37009368537 ; sigma MLE = 0.0919962490448
Inverse Hessian Matrix =
[[  7.77074808e+00  -1.92508623e-01  -1.23381164e+02  -3.89116905e-01]
 [ -1.92508623e-01   3.32190329e-02   3.12986869e+00   7.95943666e-03]
 [ -1.23381164e+02   3.12986869e+00   1.95923113e+03   6.17304090e+00]
 [ -3.89116905e-01   7.95943666e-03   6.17304090e+00   1.96469130e-02]]


In [4]:
# 1(c)
k_t = 7500000
z_t_1 = 10
r_t = 1

zstar = z_rk(r_t, k_t, a_mle)
prob = 1 - sts.norm.cdf(zstar, loc=rho_mle*z_t_1+(1-rho_mle)*mu_mle, scale=sig_mle)
print('z* =',zstar)
print('probability =', prob)

z* = 9.36943772838
probability = 0.999999967315
